# [Жёлтое такси в Нью-Йорке](https://www.coursera.org/learn/data-analysis-project/supplement/NuH2K/zhioltoie-taksi-v-n-iu-iorkie)

Нью-Йоркская комиссия по такси и лимузинам (TLC) предоставляет подробные анонимизированные данные о поездках клиентов с 2009 года. Машины, выкрашенные жёлтым, имеют право брать пассажиров на улицах в любом из пяти боро города.

Сырые данные о поездках в жёлтом такси можно найти на сайте TLC: www.nyc.gov/html/tlc/html/about/trip_record_data.shtml Эти данные разбиты на файлы по месяцам. В каждом из файлов содержится следующая информация о поездках:
* VendorID провайдер данных (одна из двух категорий)
* tpep_pickup_datetime - время начала поездки
* tpep_dropoff_datetime - время окончания поездки
* passenger_count - количество пассажиров
* trip_distance - расстояние по счётчику
* pickup_longitude &	pickup_latitude - долгота и широта точки начала поездки
* dropoff_longitude	& dropoff_latitude - долгота и широта точки окончания поездки
* RatecodeID - тип тарифа (одна из шести категорий)
* store_and_fwd_flag	- бинарный флаг, показывающий, были ли данные о поездке получены немедленно после её окончания, или какое-то время хранились в памяти автомобиля
* payment_type - способ оплаты (одна из шести категорий)
* fare_amount - стоимость поездки по счётчику
* extra - доплата за поездки в пиковые часы и ночью
* mta_tax - налог на счётчик
* tip_amount - размер чаевых
* tolls_amount - доплата за проезд по платным дорогам
* improvement_surcharge	- доплата за проезд, взимаемая с каждой поездки с января 2015
* total_amount - общая стоимость поездки

Подробнее: http://www.nyc.gov/html/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf

Задача проекта — научиться предсказывать количество поездок в ближайшие часы в каждом районе Нью-Йорка; для простоты мы определим прямоугольные районы. Для того, чтобы её решить, сырые данные необходимо агрегировать по часам и районам. Агрегированные данные будут представлять собой почасовые временные ряды с количествами поездок из каждого района.

Задачу прогнозирования таких рядов можно решать с помощью авторегрессионных моделей, прогнозируя каждый ряд независимо. Ряды явно имеют сложную сезонность — суточную, недельную и годовую, поэтому для их моделирования понадобится использовать модель ARIMA с дополнительной регрессией на внешние признаки.

Чтобы улучшить такую модель, можно попытаться учесть:

* взаимосвязи между рядами
* дополнительные параметры поездок, которые можно посчитать по исходным данным
* внешние календарные и географические признаки

В ходе выполнения проекта вы научитесь:

* работать с геоданными
* прогнозировать временные ряды сложной структуры
* строить и настраивать регрессионные модели, делающие совместные предсказания для большого количества взаимосвязанных рядов

Похожие задачи возникают на практике, если вам необходимо спрогнозировать продажи большого количества товаров в большом количестве магазинов, объём снятия денег в сети банкоматов, посещаемость разных страниц сайта и т.д.

В ходе выполнения проекта у вас будет свобода выбора хода решения, используемых средств анализа, библиотек и моделей. Также обратите внимание, что для его успешного выполнения вам нужно будет скачать и обработать от 10 до 200 гигабайт сырых данных.

In [1]:
%pylab inline

from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

Populating the interactive namespace from numpy and matplotlib


---

# [Week 5 - Прогнозирование с помощью регрессии](https://www.coursera.org/learn/data-analysis-project/peer/YhEKS/proghnozirovaniie-s-pomoshch-iu-rieghriessii)

Класс моделей ARIMA недостаточно богат для наших данных: с их помощью, например, никак нельзя учесть взаимосвязи между рядами. Это можно сделать с помощью векторной авторегрессии VARIMA, но её питоновская реализация не позволяет использовать регрессионные признаки. Кроме того, авторегрессионный подход не позволяет учитывать, например, взаимодействия между сезонными компонентами. Вы могли заметить, что форма суточных сезонных профилей в будни и выходные немного разная; явно моделировать этот эффект с помощью ARIMA не получится.

Нам нужна более сложная модель. Давайте займёмся сведением задачи массового прогнозирования рядов к регрессионной постановке!

Вам понадобится много признаков. Некоторые из них у вас уже есть — это:

* идентификатор географической зоны
* дата и время
* количество поездок в периоды, предшествующие прогнозируемому
* синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA

Кроме того, не спешите выбрасывать построенный вами на прошлой неделе прогнозы — из них может получиться хороший признак для регрессии!

Вы можете попробовать разные регрессионный модели, но хорошие результаты, скорее всего, дадут такие, которые будут позволять признакам взаимодействовать друг с другом.

Поскольку прогноз нужен на 6 часов вперёд, проще всего будет построить 6 независимых регрессионных моделей — одна для прогнозирования {y}_{T+1|T}, другая для {y}_{T+2|T} и т.д.

In [2]:
def merge_df(date_list):
    '''Объединение агрегированных данных'''
    df = pd.read_csv('data/df_regions/df_group_' + date_list[0] + '.csv', index_col=[0])
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')
    for n in date_list[1:]:
        df_merge = pd.read_csv('data/df_regions/df_group_' + n + '.csv', index_col=[0])
        df_merge.index = pd.to_datetime(df_merge.index, format='%Y-%m-%d %H:%M:%S')
        df = pd.concat([df, df_merge])
    return df

###### Формируем данные по регионам с января 2015 по июнь 2016

In [3]:
%%time
N_regions = pd.read_csv('data/df_regions/week4_region.csv', index_col=[0]).index
df = merge_df(['15-01','15-02','15-03','15-04','15-05','15-06','15-07','15-08','15-09','15-10','15-11','15-12',
                  '16-01','16-02', '16-03', '16-04', '16-05', '16-06']).iloc[:,list(N_regions-1)]

Wall time: 27.3 s


Чтобы сдать задание, выполните следующую последовательность действий.

1. Для каждой из шести задач прогнозирования сформируйте выборки. Откликом будет при всевозможных значениях TT, а признаки можно использовать следующие:
    * идентификатор географической зоны — категориальный
    * год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так
    * синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
    * сами значения прогнозов ARIMA 
    * количество поездок из рассматриваемого района в моменты времени 
    * количество поездок из рассматриваемого района в моменты времени 
    * суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц

Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании вы можете учитывать только значения yy до момента времени TT включительно.

In [4]:
def features(df):
    df_regions = pd.DataFrame(index=df.index)
    
    ## Признаки общие для всех регионов
    # календарные признаки
    df_regions[('for_all', 'X', 'year')] = df.index.year
    df_regions[('for_all', 'X', 'month')] = df.index.month
    df_regions[('for_all', 'X', 'day')] = df.index.day
    df_regions[('for_all', 'X', 'weekday')] = df.index.weekday
    df_regions[('for_all', 'X', 'hour')] = df.index.hour

    N = 15
    row = np.arange(1, df.shape[0]+1)
    for i in range(1, N+1):
        # годовая сезонность
        df_regions[('for_all', 'X', 'y_c_'+str(i))] = np.cos(row*2*np.pi*i/8766)
        df_regions[('for_all', 'X', 'y_s_'+str(i))] = np.sin(row*2*np.pi*i/8766)
        # недельная сезонность
        df_regions[('for_all', 'X', 'w_c_'+str(i))] = np.cos(row*2*np.pi*i/168)
        df_regions[('for_all', 'X', 'w_s_'+str(i))] = np.sin(row*2*np.pi*i/168)
        # дневная сезонность
        df_regions[('for_all', 'X', 'd_c_'+str(i))] = np.cos(row*2*np.pi*i/24)
        df_regions[('for_all', 'X', 'd_s_'+str(i))] = np.sin(row*2*np.pi*i/24)

    ## Признаки индивидуальные для каждого региона
    for region in df.columns:
        
        # кол-во поездок в момент времени (y.T-K)
        K = 6
        for k in range(1, K+1):
            df_regions[(region, 'X', 'T-'+str(k))] = df[region].shift(k)
        
        # кол-во поездок в момент времени (y.T-24*Kd)
        Kd = 2
        for kd in range(1, Kd+1):
            df_regions[(region, 'X', 'T-24*'+str(kd))] = df[region].shift(24*kd)
        
        # суммарное количество поездок за предшествующие полдня, сутки, неделю
        sum_N = [12, 24, 168]
        for sum_n in sum_N:
            df_regions[(region, 'X', 'sum_'+str(sum_n))] = df[region].rolling(min_periods=1, window=sum_n).sum().shift(1)
    
    
        ## Добавляем кол-во поездок по регионам
        df_regions[(region, 'y', 'real')] = df[region]
        for t in range(1,7):
            df_regions[(region, 'y', str(t))] = df[region].shift(-t)
    
    df_regions.columns = pd.MultiIndex.from_tuples(df_regions.columns, names=['region', 'X and y', 'features'])
    
    return df_regions[168:]

###### Составляем матрицу признаков для обучения

In [5]:
%%time
X_y = features(df)
X_y.to_csv('data/df_regions/week5_X.csv')

Wall time: 1min 31s


2. Разбейте каждую из шести выборок на три части:
    * обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
    * тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
    * итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016

3. Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:
    * допускала попарные взаимодействия между признаками
    * была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)

In [6]:
def predict_model(df):
    regions = np.unique((df.columns.get_level_values(level='region')))[:-1]
    len_region = regions.shape[0]
    
    start_may = '2016-04-30 23:00:00'
    end_may = '2016-05-31 17:00:00'
    predict_may = pd.DataFrame(index=df[start_may:end_may].index)

    start_june = '2016-05-31 23:00:00'
    end_june = '2016-06-30 17:00:00'
    predict_june = pd.DataFrame(index=df[start_june:end_june].index)
    
    for region in regions:
        print('Расчет предсказания для региона - {}, осталось обработать регионов - {}'.format(region, len_region-1))
        len_region -= 1
        
        X = pd.merge(df[('for_all','X')], df[(region, 'X')], left_index=True, right_index=True)
        
        for t in range(1, 7):
            print('Расчет предсказания на {} ч вперед'.format(t))
            y = df[(region, 'y', str(t))]
            ## ПРЕДСКАЗАНИЕ НА МАЙ 2016
            # разделяем данные на обучение, тестовые и итоговые
            X_train, y_train = X[:predict_may.index[0] - timedelta(hours=1)], y[:predict_may.index[0] - timedelta(hours=1)]
            X_test_may = X[start_may:end_may]
            predict_may[(region, 'y', str(t))] = df[(region, 'y', str(t))][start_may:end_may]

            # построение модели с подбором гиперпараметров
            model = Ridge()
            param_grid = [{'alpha' : np.logspace(2, 3, 50)}]
            clf = GridSearchCV(model, param_grid, cv = 5)
            best_model = clf.fit(X_train, y_train)

            # сохраняем предсказания
            predict_may[(region, 'y_predict', str(t))] = best_model.predict(X_test_may).round()   
            predict_may[(region, 'error', str(t))] = (predict_may[(region, 'y_predict', str(t))] - predict_may[(region, 'y', str(t))]).abs()

            ## ПРЕДСКАЗАНИЕ НА ИЮНЬ 2016 
            # разделяем данные на обучение, тестовые и итоговые
            X_test_june = X[start_june:end_june]
            predict_june[(region, 'y', str(t))] = df[(region, 'y', str(t))][start_june:end_june]

            # сохраняем предсказания
            predict_june[(region, 'y_predict', str(t))] = best_model.predict(X_test_june).round()   
            predict_june[(region, 'error', str(t))] = (predict_june[(region, 'y_predict', str(t))] - predict_june[(region, 'y', str(t))]).abs()
    
    predict_may.columns = pd.MultiIndex.from_tuples(predict_may.columns, names=['region', 'y', 'delta'])
    predict_june.columns = pd.MultiIndex.from_tuples(predict_june.columns, names=['region', 'y', 'delta'])
    
    return predict_may, predict_june 

###### Обучаем модели

In [7]:
%%time
y_may, y_june = predict_model(X_y)

Расчет предсказания для региона - 1075, осталось обработать регионов - 101
Расчет предсказания на 1 ч вперед


C:\Program Files\Conda_3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


Расчет предсказания на 2 ч вперед
Расчет предсказания на 3 ч вперед
Расчет предсказания на 4 ч вперед
Расчет предсказания на 5 ч вперед
Расчет предсказания на 6 ч вперед
Расчет предсказания для региона - 1076, осталось обработать регионов - 100
Расчет предсказания на 1 ч вперед


C:\Program Files\Conda_3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


Расчет предсказания на 2 ч вперед
Расчет предсказания на 3 ч вперед
Расчет предсказания на 4 ч вперед
Расчет предсказания на 5 ч вперед
Расчет предсказания на 6 ч вперед
Расчет предсказания для региона - 1077, осталось обработать регионов - 99
Расчет предсказания на 1 ч вперед


...

Расчет предсказания на 2 ч вперед
Расчет предсказания на 3 ч вперед
Расчет предсказания на 4 ч вперед
Расчет предсказания на 5 ч вперед
Расчет предсказания на 6 ч вперед
Расчет предсказания для региона - 2119, осталось обработать регионов - 1
Расчет предсказания на 1 ч вперед


C:\Program Files\Conda_3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


Расчет предсказания на 2 ч вперед
Расчет предсказания на 3 ч вперед
Расчет предсказания на 4 ч вперед
Расчет предсказания на 5 ч вперед
Расчет предсказания на 6 ч вперед
Расчет предсказания для региона - 2168, осталось обработать регионов - 0
Расчет предсказания на 1 ч вперед


C:\Program Files\Conda_3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


Расчет предсказания на 2 ч вперед
Расчет предсказания на 3 ч вперед
Расчет предсказания на 4 ч вперед
Расчет предсказания на 5 ч вперед
Расчет предсказания на 6 ч вперед
Wall time: 2h 4min 51s


4. Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:

    Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу QQ, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

In [18]:
def error(df):
    df_error = df.xs('error', level='y', axis=1)
    regions = np.unique((df.columns.get_level_values(level='region'))).shape[0]
    delta = np.unique((df.columns.get_level_values(level='delta'))).shape[0]
    return df_error.sum().sum() / (regions * df_error.shape[0] * delta)

In [19]:
%%time
y_may.to_csv('data/df_regions/week5_may.csv')
print('Ошибка качества прогнозов моделей за май составляет ', error(y_may))

Ошибка качества прогнозов моделей за май составляет  22.827361210609638
Wall time: 2.68 s


5. Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.

In [22]:
def kaggle(df):
    df_predict = df.xs('y_predict', level='y', axis=1)
    list_date = []
    list_predict = []
    for date in df_predict.index:
        for region_delta in df_predict.columns:
            list_date.append(str(region_delta[0])+'_'+str(date)[:10]+'_'+str(date.hour)+'_'+str(region_delta[1]))
            list_predict.append(df_predict[(region_delta)][date])
    return pd.DataFrame(np.array((list_date, list_predict)).T, columns=['id', 'y'])

In [23]:
%%time
y_june.to_csv('data/df_regions/week5_june.csv')
print('Ошибка качества прогнозов моделей за июнь составляет ', error(y_june))
kaggle_df = kaggle(y_june)
kaggle_df.to_csv('data/df_regions/result_june_week5.csv', index=False)

Ошибка качества прогнозов моделей за июнь составляет  20.99610128433658
Wall time: 37.4 s


6. Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

[**ссылка**](https://www.kaggle.com/c/yellowtaxi/submissions?sortBy=date&group=successful&page=1&pageSize=100), если не сработает вот скриншот

In [24]:
from IPython.display import Image # Библиотека для отображения картинок

display(Image(url='data/df_regions/kaggle_week5.png', width = 900))

7. Загрузите ноутбук в форму.

---